In [ ]:
#Problem Statement:-Suppose as a Bank Manager the target is to guess who are the churn  customer.
#Churn  customer means who had low credit score, less touch within the respective bank branches
#may be there bank account will be closed soon

#Using ANN we will solve here the classification types of Problem.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [ ]:
#Uploading the data
data=pd.read_csv('/content/Churn_Modelling.csv')
data.head(10)

In [ ]:
#Preprocessing the data
#Drop the unnecessary columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [ ]:
with open("scaler.pkl",'wb') as file:
    pickle.dump(scaler,file)

In [ ]:
from google.colab import files
#files.download('scaler.pkl')

In [ ]:
#Performing the labelEncoding Operation for Gender
label_encoder_gender=LabelEncoder()#Creating the instance with respect to the label Encoder
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head(10)

In [ ]:
#For the geography we will use the OneHotEncoder
Onehot_encoder_geography=OneHotEncoder()#Creating the instance
geo_encoder=Onehot_encoder_geography.fit_transform(data[['Geography']]).toarray()
geo_encoder

In [ ]:
#Observe the data
data.head(10)

In [ ]:
#Extracting the Geography features name using OHE
l=Onehot_encoder_geography.get_feature_names_out(['Geography'])
l

In [ ]:
#Creating Pandas DataFrame for ['Geography_France', 'Geography_Germany', 'Geography_Spain']
geo_encoded_df=pd.DataFrame(geo_encoder,columns=l)
geo_encoded_df

In [ ]:
#All the columns Except Geography
x=data.drop('Geography',axis=1)
x.columns

In [ ]:
#Combine the ONEHOT ['Geography_France', 'Geography_Germany', 'Geography_Spain'] columns with original
#data
#Combined x and geo_encoded_df
data=pd.concat([x,geo_encoded_df],axis=1)
data

In [ ]:
#Save the encoders and scalers in pickle file format
#I want to create one label_encoder_gender.pkl file in write binary mode from label_encoder_gender
with open("label_encoder_gender.pkl",'wb') as file:
    pickle.dump(label_encoder_gender,file)


with open("onehot_encoder_geo.pkl",'wb') as file:
    pickle.dump(Onehot_encoder_geography,file)

In [ ]:
#Dividing the data into dependent and independent features
X=data.drop('Exited',axis=1)#Input features
y=data['Exited']#Target Features

In [ ]:
#Spliting the data into train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
#Large value is present.I have to scalized this value
#1000=0.1
#2000=0.2
#3000=0.3
X_train
X_test

In [ ]:
#Scale the features
scaler=StandardScaler()#Creating the instance for the scaler
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
#The value has been scalized
X_train
X_test

ANN MODEL BUILD

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential# Mainly use to start Neural Network
from tensorflow.keras.layers import Dense# Use for the hidden and output layer
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#Built ANN model
model=Sequential([

                  Dense(64,activation='relu',input_dim=X_train.shape[1]),#First Hidden Layer
                  Dense(32,activation='relu'),#Second Hidden Layer
                  Dense(1,activation='sigmoid')#Output

])

In [ ]:
model.summary()

In [ ]:
#
opt=tf.keras.optimizers.Adam(learning_rate=0.001)
loss=tf.keras.losses.BinaryCrossentropy()
loss

In [ ]:
#Compile the models
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [ ]:
### Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

In [ ]:
#It's the file to connecting it with Front-end part
model.save("model.keras")

In [ ]:
## Load Tensorboard Extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
data.columns